In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('/home/maldo/projects/virtualenvs/datathon/lib/python3.7/site-packages'))
sys.path.insert(0, os.path.abspath('/home/maldo/projects/datathon'))
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *
from src.models.model import SoilClassifier
from src.constants import *
from sklearn.base import clone
from src.features.features import ImbalanceTransformer
import pandas as pd
import seaborn as sns
import math

In [2]:
df = pd.read_csv(os.path.join('../data/processed', 'train_data.csv'))
min_samples = 500
max_samples = 9000
features = ['not_transformed','squared_coordinates', 'cadastral_ordinal_encoder_onehot', 'log_area', 'savi', 'pssr', 'evi', 'evi2']
classifier = 'gradient_boosting'

In [3]:
model = SoilClassifier()
model.load('../src/experiments/results/minmax_all_gradient_boosting_500_9000_scaled.pkl')

In [9]:
pipe = clone(model.pipeline)
clsf = pipe.steps.pop()
imb = pipe.steps.pop()

In [18]:
transformed_data = pipe.fit_transform(df)

In [25]:
transformed_data = pd.DataFrame(transformed_data, columns=pipe.named_steps['features'].get_feature_names())

In [23]:
import numpy as np
def correlation_features(data, features_names):
    """Compute feature correlation.

    :param data: the matrix to compute correlations
    :type data: Spacy Sparce Matrix
    :param features_names: the features names
    :type features_names: list
    :return: The features names
    :rtype: list
    """
    df = pd.DataFrame(data, columns=features_names)
    # Compute correlation matix
    corr_matrix = df.corr(method='pearson')
    # uppper triangular matrix
    upper_mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)
    upper = corr_matrix.where(upper_mask)
    # Get correlated features
    stack_non_null = upper.stack()
    list_corr_feats = [
        (idx0, idx1, v)
        for (idx0, idx1), v
        in zip(stack_non_null.index, stack_non_null.values)
    ]
    # Sorted according its correlation value.
    list_corr_feats = sorted(list_corr_feats,
                             key=lambda x: abs(x[2]),
                             reverse=True)
    return list_corr_feats

In [24]:
correlation_features(transformed_data, features_names=pipe.named_steps['features'].get_feature_names())

[('cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_9',
  'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_Na',
  -1.0),
 ('savi__SAVI', 'evi2__EVI2', 0.9999999985840482),
 ('not_transformed__Y', 'squared_coordinates__quad_Y', 0.9999993748327856),
 ('not_transformed__X', 'squared_coordinates__quad_X', 0.999945223971755),
 ('not_transformed__Q_NIR_8_0_4',
  'not_transformed__Q_NIR_8_0_5',
  0.9939263804531154),
 ('not_transformed__Q_R_4_0_5',
  'not_transformed__Q_R_4_0_6',
  0.993743326231206),
 ('not_transformed__Q_NIR_8_0_3',
  'not_transformed__Q_NIR_8_0_4',
  0.9936077136417464),
 ('not_transformed__Q_R_4_0_4',
  'not_transformed__Q_R_4_0_5',
  0.9933661141995213),
 ('not_transformed__Q_NIR_8_0_5',
  'not_transformed__Q_NIR_8_0_6',
  0.9932793752819997),
 ('not_transformed__Q_NIR_8_0_6',
  'not_transformed__Q_NIR_8_0_7',
  0.9928150526823),
 ('not_transformed__Q_NIR_8_0_2',
  'not_transformed__Q_NIR_8_0_3',
  0.9919414416274379),
 ('not_transformed__Q_R_4_0_6',
  'not_tran

In [41]:
corr_matrix = transformed_data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

In [43]:
len(to_drop)

44

In [44]:
dropped_highly = transformed_data.copy()
dropped_highly.drop(columns=to_drop, inplace=True)
dropped_highly.columns

Index(['not_transformed__X', 'not_transformed__Y',
       'not_transformed__Q_R_4_0_0', 'not_transformed__Q_R_4_0_1',
       'not_transformed__Q_R_4_1_0', 'not_transformed__Q_G_3_0_0',
       'not_transformed__Q_B_2_0_0', 'not_transformed__Q_B_2_0_1',
       'not_transformed__Q_B_2_1_0', 'not_transformed__Q_NIR_8_0_0',
       'not_transformed__Q_NIR_8_0_1', 'not_transformed__GEOM_R1',
       'not_transformed__GEOM_R2', 'not_transformed__GEOM_R3',
       'not_transformed__GEOM_R4', 'not_transformed__CONTRUCTIONYEAR',
       'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_9',
       'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_8',
       'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_7',
       'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_6',
       'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_5',
       'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_4',
       'cadastral_ordinal_encoder_onehot__CADASTRALQUALITYID_3',
       'cadastral_ordin

In [45]:
dropped_highly.shape

(82584, 32)

In [46]:
imbt = ImbalanceTransformer(min_samples=min_samples, max_samples=max_samples)

In [47]:
X,y = imbt.fit_resample(dropped_highly, df[TARGET_FEATURE])

In [48]:
X.shape

(19597, 32)

In [49]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X, y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [55]:
from sklearn.metrics import *
def calculate_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted'),
    }

In [56]:
calculate_metrics(y, gb.predict(X))

{'accuracy': 0.6911772210032148,
 'balanced_accuracy': 0.5637470461166266,
 'precision_macro': 0.6937733013392038,
 'precision_weighted': 0.6850614024300065,
 'recall_macro': 0.5637470461166266,
 'recall_weighted': 0.6911772210032148,
 'f1_macro': 0.5937363344282583,
 'f1_weighted': 0.6630916119308298}

In [59]:
model.evaluate(df, df[TARGET_FEATURE])

In [60]:
model.metrics

{'test': {'accuracy': 0.8642230940618038,
  'custom_accuracy': 0.6864556049202611,
  'balanced_accuracy': 0.563214013748093,
  'precision_macro': 0.5281174993732494,
  'precision_weighted': 0.8856401913319055,
  'recall_macro': 0.563214013748093,
  'recall_weighted': 0.8642230940618038,
  'f1_macro': 0.5232338727039931,
  'f1_weighted': 0.8710102074881987}}